<a href="https://colab.research.google.com/github/crodier1/machine_learning_deep_learning/blob/main/RAG_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instructions
*   You may use the Deepseek or OpenAI URL
*   Provide your own API key
*   Ask a question about any article in the sample
*   The agent remebers your questions. You may ask follow up questions.



In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import requests
import os
from openai import OpenAI

# Precompute embeddings (run once and save)
def precompute_embeddings(df, text_column='text'):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    texts = df[text_column].tolist()
    embeddings = model.encode(texts, show_progress_bar=True)
    df['embeddings'] = list(embeddings)
    return df

# Load your DataFrame (example)
df_small = pd.read_csv('https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv').sample(100)
df_small = precompute_embeddings(df_small, 'text')
df_small.to_pickle('df_small_embeddings.pkl')  # Save for later

class QA_Agent:
    def __init__(self, df_path, api_key, url):
        self.df = pd.read_pickle(df_path)
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.client = OpenAI(api_key=api_key, base_url=url)
        self.messages = [
            {'role': 'system', 'content': "Answer ONLY using the provided context. If unsure, say 'I don't know the answer.'"}
        ]



    def _get_most_relevant_context(self, question, threshold=0.3):
        question_embed = self.embedding_model.encode([question])
        df_embeddings = np.stack(self.df['embeddings'].values)
        similarities = cosine_similarity(question_embed, df_embeddings)[0]
        max_idx = np.argmax(similarities)
        return self.df.iloc[max_idx]['text'] if similarities[max_idx] > threshold else None

    def ask(self, question):
        context = self._get_most_relevant_context(question)
        if not context:
            return "I don't know the answer."

        self.messages.append({'role': 'user', 'content': f"Context:\n{context}\n\nQuestion: {question}"})

        response = self.__complete()
        answer = response.choices[0].message.content
        self.messages.append({'role': 'assistant', 'content': answer})

        return answer

    def __complete(self):
      return self.client.chat.completions.create(
          model='deepseek-chat',
          messages=self.messages,
          temperature=0,
          max_tokens=200
      )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
print(df_small.sample(1)['text'].values[0])

Merritt close to indoor 400m mark

Teenager LaShawn Merritt ran the third fastest indoor 400m of all time at the Fayetteville Invitational meeting.

The world junior champion clocked 44.93 seconds to finish well clear of fellow American Bershawn Jackson in Arkansas. Only Michael Johnson has gone quicker, setting the world record of 44.63secs in 1995 and running 44.66secs in 1996. Kenyan Bernard Lagat missed out on the world record by 1.45secs as he ran the third quickest indoor mile ever to beat Canada's Nate Brannen by almost 10secs. The Olympic silver medallist's time of three minutes 49.89secs was inferior only to the 1997 world record of Moroccan Hicham El Guerrouj and former world record holder Eamonn Coghlan of Ireland's 3:49.78. Lagat was on course to break El Guerrouj's record through 1200m but could not maintain the pace over the final 400m. Ireland's

continued his excellent form by winning a tight 3,000m in 7:40.53. Cragg, who recently defeated Olympic 10,000m champion Kenen

In [ ]:
df_path = 'df_small_embeddings.pkl'
api_key = '[put your deepseek or OpenAPi key here]'
# You may also use the openAI URL
url = "https://api.deepseek.com"
agent = QA_Agent(df_path, api_key, url)

In [ ]:
while True:
    question = input("Ask a question (or 'exit' to quit): ")
    if question.lower() == 'exit':
        break
    answer = agent.ask(question)
    print("Answer:", answer)

Ask a question (or 'exit' to quit): Where did LaShawn Merritt run?
Answer: LaShawn Merritt ran at the Fayetteville Invitational meeting in Arkansas.
Ask a question (or 'exit' to quit): How fast did he run?
Answer: LaShawn Merritt ran the indoor 400m in **44.93 seconds**.
Ask a question (or 'exit' to quit): Why is the sky blue?
Answer: I don't know the answer.
Ask a question (or 'exit' to quit): quit
Answer: I don't know the answer.
Ask a question (or 'exit' to quit): exit
